## Way 2: SQL style

In [1]:
# Call packages
import numpy as np
import pandas as pd
import datetime as dt

In [2]:
# Read data
prices = pd.read_csv('prices.csv')
sales = pd.read_csv('sales.csv')

In [3]:
## Standardize datetime type
prices['updated_at'] = pd.to_datetime(prices['updated_at'])
sales['ordered_at'] = pd.to_datetime(pd.to_datetime(sales['ordered_at']).dt.strftime('%Y-%m-%d %H:%M:%S'))

In [4]:
# #### Ý tưởng
# Dùng SQL: LEAD(updated_at,1, NOW()) 
#           OVER(PARTITION BY product_id ORDER BY product_id, updated_at) AS old_date

# Tạo ra bảng kiểu như:   --id---old_price---new_price---old_date---new_date

# Nếu Sales.time nằm trong khoảng từ old_date đến new_date thì price sẽ là new_price
# Nếu không có time nào như thế tức là đơn hàng chưa trải qua lần update price nào, price sẽ là
# old_price của lần update sớm nhất

In [5]:
## Create a reference-price table
prices = prices.sort_values(['product_id', 'updated_at'])
prices['new_date'] = prices.groupby('product_id')\
                ['updated_at'].shift(-1, fill_value= dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

In [6]:
## rename for easily using
prices = prices.rename(columns= {'updated_at': 'old_date'})

In [7]:
## see what we have:
prices.head()

,product_id,old_price,new_price,old_date,new_date
0,64,270000,239000,2018-09-10 16:37:00,2022-10-16 16:38:08
4,3954203,68800,60000,2018-09-10 16:32:00,2018-09-11 11:54:00
1,3954203,60000,64000,2018-09-11 11:54:00,2018-09-15 03:49:00
3,3954203,64000,60500,2018-09-15 03:49:00,2018-09-17 22:59:00
2,3954203,60500,57500,2018-09-17 22:59:00,2022-10-16 16:38:08


In [8]:
## Define function to return price
def return_price(id, time):
    for i in prices.index:
#         if time between old_date & new_date, price is new_date
        if  (id == prices.product_id[i]) & ((time >= prices.old_date[i]) & (time < prices.new_date[i])):
            price_return = prices.new_price[i]
            break    
#         if not, dueto arranged dataframe, price will be the old_price of the earliest updated price time
        elif (id == prices.product_id[i]) & (time <= prices.old_date[i]):  
            price_return = prices.old_price[i]
            break
    return price_return

In [9]:
## A similar problem as Way1
sales['price'] = pd.Series(dtype='int64')

for i in sales.index:
         sales.price[i] = return_price(sales.product_id[i], sales.ordered_at[i])

C:\Users\Admin\AppData\Local\Temp\ipykernel_3256\3144351881.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales.price[i] = return_price(sales.product_id[i], sales.ordered_at[i])


In [10]:
# price_returned_array = pd.array([]).astype('int64')
# for i in sales.index:
#     price_returned = return_price(sales.product_id[i], sales.ordered_at[i])
#     price_returned_array = np.append(price_returned_array, price_returned)
    
# sales['price'] = price_returned_array

In [11]:
sales['revenue'] = sales.quantity_ordered * sales.price

In [12]:
sales.pivot_table(
    values= 'revenue',
    index= ['product_id', 'price'],
    aggfunc= sum
)

revenue
product_id price              
64         239000.0   956000.0
3954203    57500.0     57500.0
           60000.0    180000.0
           64000.0    640000.0
3998909    15500.0     15500.0
           16500.0    231000.0
           17000.0     34000.0
4085861    52000.0   1040000.0
           53500.0   2140000.0
           58000.0   2204000.0
           60000.0    180000.0
           62500.0   1812500.0
           67000.0    871000.0

### Few note:
e thấy cách này nhìn gần gũi hơn vì nó giống sql nhưng mà với lượng data lớn thì việc sort, rồi update dataframe nhiều lần như vậy có vẻ không hay, em chỉ hoàn thiện phương án này vì e trót làm dở rồi thôi ạ:)), e thấy cách 1 của e vẫn hay hơn, k biết em nhận định vậy có đúng k